<figure>
  <IMG SRC="input/TUM.png" WIDTH=250 ALIGN="right">
</figure>

# Sequence Analysis with Biopython
    
*David B. Blumenthal, Olga Lazareva*

## Biopython

- A very comprehensive collection of tools for biological computation in Python.
- Today, we'll cover only a very small subset of the available functionality.
- For more information, read the [documentation](https://biopython.org/) or have a look at Peter Cock's [Introduction to Biopython](https://github.com/peterjc/biopython_workshop) course.

### Biopython imports

In [2]:
import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW

---
## Very first steps

### Reading FASTA files via `SeqIO.parse()` and `SeqIO.read()`

- `SeqIO.parse()` returns iterator over all sequences contained in the record.
- `SeqIO.read()` can be used to return the unique sequence contained in FASTA files that contain exactly one record. 

In [4]:
fasta_file = 'input/unknown_sequence.fasta'
file_format = 'fasta'
records = SeqIO.parse(fasta_file, file_format)
for record in records:
    print(record)
record = SeqIO.read(fasta_file, file_format)
print(record)

ID: Unknown_sequence
Name: Unknown_sequence
Description: Unknown_sequence
Number of features: 0
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA')
ID: Unknown_sequence
Name: Unknown_sequence
Description: Unknown_sequence
Number of features: 0
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA')


### Assessing the lenght of the sequence

In [18]:
print(f'The length of the sequence is {len(record)} BP.')

The lenght of the sequence is 29903 BP.


### Assessing the GC content of the sequence

In [24]:
print(f'The GC content of the sequence is {GC(record.seq):.2f} %.')

The GC content of the sequence is 37.97 %.


---
## Sequence alignment with BLAST

- Use `Bio.Blast.NCBIWWW.qblast()` to run BLAST via the online API provided by NCBI.

### Use `help(FUNCTIONNAME)` to show information about usage

- This works for all well-written Python functions (and also with Python classes).
- If you want to learn more, google for **docstring**.

In [25]:
help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None)
    BLAST search using NCBI's

### Run the query and write the result in an XML file (might take a while, we are doing an online search)

In [37]:
%%time
result_handle = NCBIWWW.qblast(program='blastn', database='nt', sequence=record.seq)
with open('input/blast_unknown_sequence.xml', 'w') as fp:
    fp.write(result_handle.read())
    result_handle.close()

CPU times: user 714 ms, sys: 185 ms, total: 900 ms
Wall time: 50min 40s


### Read the BLAST result and print the first high-scoring pair

In [89]:
blast_qresult = SearchIO.read('input/blast_unknown_sequence.xml', 'blast-xml')
first_hit = blast_qresult[0]
print(f'Description: {first_hit.description}')
first_hsp = first_hit[0] 
print(first_hsp)

Description: Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
      Query: No definition line
        Hit: gi|1798174254|ref|NC_045512.2| Severe acute respiratory syndrome...
Query range: [0:29903] (1)
  Hit range: [0:29903] (1)
Quick stats: evalue 0; bitscore 53927.40
  Fragments: 1 (29903 columns)
     Query - ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATC~~~AAAAA
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATC~~~AAAAA


---
## Extract meta-data from NCBI Entrez

### Extract NCBI ID from result and tell NCBI who you are

In [93]:
NCBI_id = first_hit.id.split('|')[3]
Entrez.email = 'david.blumenthal@wzw.tum.de'

### Query NCBI Entrez

In [96]:
genbank_handle = Entrez.efetch(db='nucleotide', id=NCBI_id, retmode='text', rettype='gb')
genbank_record = SeqIO.read(genbank_handle, 'genbank')

In [98]:
print(genbank_record)

ID: NC_045512.2
Name: NC_045512
Description: Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Database cross-references: BioProject:PRJNA485481
Number of features: 57
/molecule_type=ss-RNA
/topology=linear
/data_file_division=VRL
/date=18-JUL-2020
/accessions=['NC_045512']
/sequence_version=2
/keywords=['RefSeq']
/source=Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
/organism=Severe acute respiratory syndrome coronavirus 2
/taxonomy=['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
/references=[Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Programmed ribosomal frameshifting in decoding the SARS-CoV genome', ...), Reference(title='The structure of a rigorously conserved RNA element within the SARS virus genome', ...), Reference(title="A phyl

### What kind of virus are we looking at?

In [101]:
print(f"Virus type: {genbank_record.annotations['molecule_type']} virus.")

Virus type: ss-RNA virus.


### What is its taxonomy?

In [106]:
print(f'Virus taxonomy: {" >> ".join(genbank_record.annotations["taxonomy"])}')

Virus taxonomy: Viruses >> Riboviria >> Orthornavirae >> Pisuviricota >> Pisoniviricetes >> Nidovirales >> Cornidovirineae >> Coronaviridae >> Orthocoronavirinae >> Betacoronavirus >> Sarbecovirus


### Relevant publications or labs

In [108]:
for reference in genbank_record.annotations['references']:
    print(reference)

location: [0:29903]
authors: Wu,F., Zhao,S., Yu,B., Chen,Y.M., Wang,W., Song,Z.G., Hu,Y., Tao,Z.W., Tian,J.H., Pei,Y.Y., Yuan,M.L., Zhang,Y.L., Dai,F.H., Liu,Y., Wang,Q.M., Zheng,J.J., Xu,L., Holmes,E.C. and Zhang,Y.Z.
title: A new coronavirus associated with human respiratory disease in China
journal: Nature 579 (7798), 265-269 (2020)
medline id: 
pubmed id: 32015508
comment: Erratum:[Nature. 2020 Apr;580(7803):E7. PMID: 32296181]

location: [13475:13503]
authors: Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F., Atkins,J.F. and Howard,M.T.
title: Programmed ribosomal frameshifting in decoding the SARS-CoV genome
journal: Virology 332 (2), 498-510 (2005)
medline id: 
pubmed id: 15680415
comment: 

location: [29727:29768]
authors: Robertson,M.P., Igel,H., Baertsch,R., Haussler,D., Ares,M. Jr. and Scott,W.G.
title: The structure of a rigorously conserved RNA element within the SARS virus genome
journal: PLoS Biol. 3 (1), e5 (2005)
medline id: 
pubmed id: 15630477
comment: 

l

---
## Protein level analysis

### Get the protein-coding sequences (CDSs)

In [116]:
CDSs = [feature for feature in genbank_record.features if feature.type == "CDS"]

protein_seq = Seq(CDSs[0].qualifiers['translation'][0])
protein_seq

Seq('ORF1ab')

### Have a closer look at the first CDS

In [139]:
first_cds = CDSs[0]
print(f'Gene name of first CDS: {first_cds.qualifiers["gene"][0]}.') # Gene name of first CDS.
protein_seq = Seq(first_cds.qualifiers['translation'][0])            # Get amino acid sequence of first CDS.
print(first_cds)                                                     # Print first CDS.

Gene name of first CDS: ORF1ab.
type: CDS
location: join{[265:13468](+), [13467:21555](+)}
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GeneID:43740578']
    Key: gene, Value: ['ORF1ab']
    Key: locus_tag, Value: ['GU280_gp01']
    Key: note, Value: ['pp1ab; translated by -1 ribosomal frameshift']
    Key: product, Value: ['ORF1ab polyprotein']
    Key: protein_id, Value: ['YP_009724389.1']
    Key: ribosomal_slippage, Value: ['']
    Key: translation, Value: ['MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVE

### Write first CDS to FASTA file

In [148]:
seq_record = SeqRecord(seq=Seq(first_cds.qualifiers['translation'][0]),
                      id=first_cds.qualifiers['protein_id'][0],
                      name=first_cds.qualifiers['product'][0],
                      dbxrefs=first_cds.qualifiers['db_xref'])
SeqIO.write(seq_record, f'output/{first_cds.qualifiers["protein_id"][0]}.fasta', 'fasta')

1

---
## <a name="ex1"></a>Exercise 1

- The file `input/dna_sequences.fasta` contains further DNA sequences.
- Analyze some of these sequences with the techniques presented today.
- Repeat this with your favourite FASTA file (if you have any).

<a href="#ex1sol">Solution for Exercise 1</a>

---
## Solutions for exercises

<a name="ex1sol">Solution for Exercise 1</a>

In [149]:
fasta_file = 'input/dna_sequences.fasta'
file_format = 'fasta'
records = [record for record in SeqIO.parse(fasta_file, file_format)]

In [150]:
records[0]

SeqRecord(seq=Seq('TGGGGAATCTTCCGCAATGGGCGAAAGCCTGACGGAGCAACGCCGCGTGAGTGA...ACA'), id='35fd41579b5eec27ec14c400c586c480', name='35fd41579b5eec27ec14c400c586c480', description='35fd41579b5eec27ec14c400c586c480', dbxrefs=[])

In [151]:
result_handle = NCBIWWW.qblast('blastn', 'nt', records[0].seq)
with open('output/dna_sequence_0.xml', 'w') as fp:
    fp.write(result_handle.read())
    result_handle.close()

In [169]:
blast_qresult = SearchIO.read('output/dna_sequence_0.xml', 'blast-xml')
first_hit = blast_qresult[0]
print(f'Description: {first_hit.description}')
first_hsp = first_hit[0] 

Description: Dialister invisus strain KCOM 3293 (=ChDC P001-Td3) 16S ribosomal RNA gene, partial sequence


In [154]:
NCBI_id = first_hit.id.split('|')[3]
genbank_handle = Entrez.efetch(db='nucleotide', id=NCBI_id, retmode='text', rettype='gb')
genbank_record = SeqIO.read(genbank_handle, 'genbank')

In [162]:
print(genbank_record)

ID: MT471996.1
Name: MT471996
Description: Dialister invisus strain KCOM 3293 (=ChDC P001-Td3) 16S ribosomal RNA gene, partial sequence
Number of features: 2
/molecule_type=DNA
/topology=linear
/data_file_division=BCT
/date=19-MAY-2020
/accessions=['MT471996']
/sequence_version=1
/keywords=['']
/source=Dialister invisus
/organism=Dialister invisus
/taxonomy=['Bacteria', 'Firmicutes', 'Negativicutes', 'Veillonellales', 'Veillonellaceae', 'Dialister']
/references=[Reference(title='Isolation of oral bacteria from a Korean population', ...), Reference(title='Direct Submission', ...)]
/comment=Sequences were screened for chimeras by the submitter using
Lasergene 7.0.
/structured_comment=OrderedDict([('Assembly-Data', OrderedDict([('Sequencing Technology', 'Sanger dideoxy sequencing')]))])
Seq('GACGAACGCTGGCGGCGTGCTTAACACATGCAAGTCGAACGAAAAGAGGGGAAG...GTG')


<a href="#ex1">Back to Exercise 1</a>